In [2]:
import re
import datetime
from urlparse import urlparse
from datetime import datetime
from datetime import timedelta
# import time
from math import exp
from pyspark.sql import Row
from pyspark import SparkContext
import sys
import os
import csv
import time

import pygal
from pygal.style import Style
import numpy as np
import statsmodels.api as sm # recommended import according to the docs
#import matplotlib.pyplot as plt
from math import exp
from pylab import *
import socket
import collections
import pygeoip
import itertools
from pygal.style import Style

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))


def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)

    if match is None:
        return (logline, 0)
    size_field = match.group(7)
    if size_field == '-':
        size = long(0)
    else:
        size = long(match.group(7))

    return (Row(
        unique_id     = match.group(1),
        depth         = match.group(3),
        response_code = int(match.group(5)),
        content_size  = size,
        url           = match.group(9),
        cookies       = match.group(11),
        objecttype    = match.group(13),
        host          = match.group(15),
        ip_address    = match.group(17),
        asn_number    = match.group(19),
        distinct_asn_number    = match.group(21),
        InternalObjectCount = match.group(23),
        ExternalObjectCount = match.group(25),
        NumberOfuniqueExternalSecondlevelSites = match.group(27),     
        start_time    = match.group(29),
        end_time      = match.group(31)
    ), 1)


APACHE_ACCESS_LOG_PATTERN = '(\S+)(\t)(\d{1})(\t)(\d{3})(\t)(\w+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)(\S+)(\t)([0-5]?[0-9]:[0-5]?[0-9]:[0-5]?[0-9].\d{6})(\t)([0-5]?[0-9]:[0-5]?[0-9]:[0-5]?[0-9].\d{6})'
logFile = os.path.join('/home/soumya/Documents/thesis_new/webcrawlTest/output_final1.csv')
timestr = time.strftime("%Y%m%d")
folder_name='result_graphs_'+timestr
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)
print logFile


#sc=SparkContext("local", "pyspark_output.py")
#sc=SparkContext()

def parseLogs():
    """ Read and parse log file """
    parsed_logs = (sc.textFile(logFile,use_unicode=False).map(parseApacheLogLine).cache())

    access_logs = (parsed_logs
                   .filter(lambda s: s[1] == 1)
                   .map(lambda s: s[0])
                   .cache())

    failed_logs = (parsed_logs
                   .filter(lambda s: s[1] == 0)
                   .map(lambda s: s[0]).cache())
    failed_logs_count = failed_logs.count()
    print "failed",failed_logs_count
    if failed_logs_count > 0:
        print 'Number of invalid logline: %d' % failed_logs.count()
        for line in failed_logs.take(20):
            print 'Invalid logline: %s' % line

    print 'Read %d lines, successfully parsed %d lines, failed to parse %d lines' % (parsed_logs.count(), access_logs.count(), failed_logs.count())
    #print 'Read %d lines, successfully parsed %d lines' % (parsed_logs.count(), access_logs.count())
    return parsed_logs, access_logs,failed_logs


parsed_logs, access_logs, failed_logs = parseLogs()

/home/soumya/Documents/thesis_new/webcrawlTest/output_final1.csv
failed 0
Read 7688751 lines, successfully parsed 7688751 lines, failed to parse 0 lines


In [2]:
#sites Crawled from top 100,000 sites
unique_id=access_logs.map(lambda log: log.unique_id).cache()
unique_id_values=set(unique_id.collect())
print len(unique_id_values)

61270


In [10]:
from netaddr import *
ipaddressRDD=access_logs.map(lambda log:(log.ip_address,log.host)).cache()
ipaddressRDDProperRDD=ipaddressRDD.map(lambda (ipadress,host):(ipadress.split(';'),host.split(';')[-1]))
ipaddressRDDwithoutNullRDD=ipaddressRDDProperRDD.filter(lambda (x,y):'-' not in x and '-' not in y).saveAsTextFile("IPHOSTMAP")
#allIPHostswithSecondlevelDomainRDD=ipaddressRDDwithoutNullRDD.map(getsecondleveldomainValue)
#allIPHostswithSecondlevelDomainRDD.take(10)
#allIPHostRDD = allIPHostswithSecondlevelDomainRDD.groupBy(lambda word: word[0])
#print allIPHostRDD.take(10)

In [3]:
hostRDD=access_logs.map(lambda log:(log.host,1)).cache()
hostProperRDD=hostRDD.map(lambda (host,value):(host.split(';'),1))
hostwithoutNullRDD=hostProperRDD.filter(lambda (x,y):'-' not in x)
#print hostwithoutNullRDD.take(10)
def getsecondleveldomain(url):
    with open("effective_tld_names.dat") as tld_file:
        tlds = [line.strip() for line in tld_file if line[0] not in "/\n"]
    url_elements = urlparse(url)[1].split('.')
    for i in range(-len(url_elements), 0):
        last_i_elements = url_elements[i:]
        candidate = ".".join(last_i_elements) # abcde.co.uk, co.uk, uk
        wildcard_candidate = ".".join(["*"] + last_i_elements[1:]) # *.co.uk, *.uk, *
        exception_candidate = "!" + candidate
        # match tlds: 
        if (exception_candidate in tlds):
            return url_elements[i:][0]
        if (candidate in tlds or wildcard_candidate in tlds):
            return url_elements[i-1:][0]
def getsecondleveldomainValue(value):
    urls=value[0]
    for domain in urls :
        if domain.startswith('www.'):
            domain = domain.replace("www","")
        if domain.endswith('.'):
            domain=domain[:-1]
        if not domain.startswith('http://'):
            domain = 'http://%s' % domain
        secondlevelurl=str(getsecondleveldomain(domain))
        return (secondlevelurl,value[1])
def some_function(a, b):
    some_result=a+b
    return some_result
allHostswithSecondlevelDomainRDD=hostwithoutNullRDD.map(getsecondleveldomainValue)
hostcheckRDD=allHostswithSecondlevelDomainRDD.reduceByKey(some_function).saveAsTextFile("hostlist")
#print hostcheckRDD.takeOrdered(100, key=lambda x: -x[1])

In [10]:
from operator import itemgetter, attrgetter, methodcaller
objectRDD=access_logs.map(lambda log:(log.objecttype,1)).cache()
objectProperRDD=objectRDD.filter(lambda (x,y):'-' not in x)
objectProperRDDCount=objectProperRDD.reduceByKey(some_function)
collectobject=objectProperRDDCount.collect()
hostwithObjectRDD=access_logs.map(lambda log:(log.objecttype,log.host)).cache()
hostWithObjectProperRDD=hostwithObjectRDD.map(lambda (objecttype,host):(objecttype,host.split(';')))
hostObjectwithoutNullRDD=hostWithObjectProperRDD.filter(lambda (x,y):'-' not in y)
for value in collectobject:
    print "object type",value[0]
    hostwithObject=hostObjectwithoutNullRDD.filter(lambda (x,y):value[0] in x).map(lambda (x,y):(y,1)).cache()
    allHostObjectswithSecondlevelDomainRDD=hostwithObject.map(getsecondleveldomainValue)
    hostobjectCheckRDD=allHostObjectswithSecondlevelDomainRDD.reduceByKey(some_function)#.saveAsTextFile(value[0])
    if len(hostobjectCheckRDD.collect()) >0:
        print hostobjectCheckRDD.takeOrdered(100, key=lambda x: -x[1])

object type body
[('servimg', 3), ('yahoodns', 3), ('CyberPulse', 1), ('ihya', 1), ('iransetup', 1), ('gunadarma', 1), ('usenetrevolution', 1), ('ustatik', 1), ('onlinegames', 1), ('songslover', 1), ('countrycallingcodes', 1), ('africav', 1), ('dechile', 1), ('oewheelsllc', 1), ('smileytraffic', 1), ('baseball-almanac', 1), ('kids-in-mind', 1), ('edgekey', 1), ('byteark', 1), ('listwire', 1), ('h-rez', 1), ('v2cdn', 1), ('vin', 1), ('hockeydb', 1), ('munjanara', 1)]
object type em
[('google', 124), ('youku', 18), ('edgesuite', 13), ('powercdn', 13), ('ourwebpic', 11), ('kunlunpi', 11), ('kunlungr', 10), ('edgecastcdn', 10), ('sanliurfa', 10), ('akamai', 9), ('tingsea', 8), ('sensocriticopb', 7), ('ourwebcdn', 7), ('d1ge0kk1l5kms0', 6), ('wscdns', 6), ('yijintong', 5), ('yunjiasu-cdn', 5), ('ln2car', 5), ('rolexforums', 4), ('catolenews', 4), ('cdn20', 4), ('incapdns', 4), ('87pk', 4), ('amazonaws', 4), ('deepika', 4), ('fhldns', 4), ('cdntip', 4), ('newjamendo', 4), ('lecloud', 4), ('a